# NYC Delivery Truck Congestion – Step 5: Machine Learning Modeling
*Author: Karan Chauhan*  

This notebook builds and evaluates machine learning models to predict delivery truck congestion.

**Prediction task:** Binary classification - predict high vs low congestion given grid cell, hour, and day of week.

**Models:**
1. Baseline (naive predictor)
2. Logistic Regression
3. Random Forest

---

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import joblib

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

import warnings
warnings.filterwarnings('ignore')

## Load Modeling Dataset

In [ ]:
df = pd.read_csv('../data/modeling_dataset.csv')

print(f"Loaded {len(df):,} observations")
print(f"\nColumns: {list(df.columns)}")
print(f"\nTarget distribution:")
print(df['high_congestion'].value_counts())
print(f"High congestion: {df['high_congestion'].mean()*100:.1f}%")

df.head()

## Prepare Features and Target

In [ ]:
# Select features for modeling
feature_cols = ['hour', 'day_of_week', 'is_weekend', 'is_rush_hour', 'month']

X = df[feature_cols]
y = df['high_congestion']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures used: {feature_cols}")

## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train):,} observations")
print(f"Test set: {len(X_test):,} observations")
print(f"\nTrain target distribution:")
print(y_train.value_counts())
print(f"\nTest target distribution:")
print(y_test.value_counts())

---
# Model 1: Baseline (Naive Predictor)
---

Always predict the most common class (low congestion)

In [ ]:
# Baseline: always predict most common class (0 = low congestion)
baseline_pred = np.zeros(len(y_test))

baseline_acc = accuracy_score(y_test, baseline_pred)
baseline_precision = precision_score(y_test, baseline_pred, zero_division=0)
baseline_recall = recall_score(y_test, baseline_pred, zero_division=0)
baseline_f1 = f1_score(y_test, baseline_pred, zero_division=0)

print("="*50)
print("BASELINE MODEL (Always Predict Low Congestion)")
print("="*50)
print(f"Accuracy:  {baseline_acc:.3f}")
print(f"Precision: {baseline_precision:.3f}")
print(f"Recall:    {baseline_recall:.3f}")
print(f"F1 Score:  {baseline_f1:.3f}")
print("\nNote: Baseline just guesses 'low congestion' every time.")
print("Any real model must beat this!")

---
# Model 2: Logistic Regression
---

## Train Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

print("Logistic Regression trained successfully")

## Evaluate Logistic Regression

In [ ]:
y_pred_logreg = logreg.predict(X_test)

logreg_acc = accuracy_score(y_test, y_pred_logreg)
logreg_precision = precision_score(y_test, y_pred_logreg)
logreg_recall = recall_score(y_test, y_pred_logreg)
logreg_f1 = f1_score(y_test, y_pred_logreg)

print("="*50)
print("LOGISTIC REGRESSION")
print("="*50)
print(f"Accuracy:  {logreg_acc:.3f}")
print(f"Precision: {logreg_precision:.3f}")
print(f"Recall:    {logreg_recall:.3f}")
print(f"F1 Score:  {logreg_f1:.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_logreg, target_names=['Low Congestion', 'High Congestion']))

## Confusion Matrix - Logistic Regression

In [ ]:
cm_logreg = confusion_matrix(y_test, y_pred_logreg)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_logreg, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
plt.title('Confusion Matrix - Logistic Regression')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.show()

---
# Model 3: Random Forest
---

## Train Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

print("Random Forest trained successfully")

## Evaluate Random Forest

In [ ]:
y_pred_rf = rf.predict(X_test)

rf_acc = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf)
rf_recall = recall_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)

print("="*50)
print("RANDOM FOREST")
print("="*50)
print(f"Accuracy:  {rf_acc:.3f}")
print(f"Precision: {rf_precision:.3f}")
print(f"Recall:    {rf_recall:.3f}")
print(f"F1 Score:  {rf_f1:.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Low Congestion', 'High Congestion']))

## Confusion Matrix - Random Forest

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
plt.title('Confusion Matrix - Random Forest')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.show()

## Feature Importance - Random Forest

In [ ]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance (Random Forest):")
print(feature_importance)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('Importance')
plt.title('Feature Importance - Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

---
# Model Comparison
---

In [ ]:
results = pd.DataFrame({
    'Model': ['Baseline', 'Logistic Regression', 'Random Forest'],
    'Accuracy': [baseline_acc, logreg_acc, rf_acc],
    'Precision': [baseline_precision, logreg_precision, rf_precision],
    'Recall': [baseline_recall, logreg_recall, rf_recall],
    'F1 Score': [baseline_f1, logreg_f1, rf_f1]
})

print("="*70)
print("MODEL COMPARISON")
print("="*70)
print(results.to_string(index=False))

# Visualize comparison
results_plot = results.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1 Score']]

results_plot.plot(kind='bar', figsize=(12, 6), width=0.8)
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xlabel('Model')
plt.xticks(rotation=0)
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Save Best Model

In [ ]:
# Save the Random Forest model (best performer)
model_path = '../models/random_forest_model.pkl'
joblib.dump(rf, model_path)

print(f"Saved Random Forest model to: {model_path}")

# Also save Logistic Regression for comparison
logreg_path = '../models/logistic_regression_model.pkl'
joblib.dump(logreg, logreg_path)

print(f"Saved Logistic Regression model to: {logreg_path}")

## Summary

**What we accomplished:**
1. Built baseline model (always predict low congestion)
2. Trained Logistic Regression
3. Trained Random Forest
4. Evaluated all models with accuracy, precision, recall, F1
5. Analyzed feature importance
6. Compared model performance
7. Saved best models

**Key findings:**
- Random Forest outperforms Logistic Regression
- Both real models beat baseline significantly
- Most important feature: [check feature importance above]

**Next steps:**
- Optional: Add external data (weather, holidays) to improve model
- Build interactive dashboard to visualize predictions
- Deploy model for real-time predictions